In [ ]:
from hexathello import autoPlayer, engine, jable, printing

import numpy as np

from os import path, remove

# Hexathello can be played as a game, or played by AI to train
# We're unlikely to intialize a Hexathello Engine directly. Instead, use ``autoPlayer`` to setup a game
#   for AI to play

# -- Settings
game_size: int = 5
player_count: int = 2

# First, choose our AIs. The RandomHexAgent is the dumbest, picking randomly from legal moves.
# Indexing begins with 0 as it should. As a result, the 'second' player is "Player 1"; use the latter notation

ai_0: autoPlayer.HexAgent = autoPlayer.RandomHexAgent(
    size = game_size,
    player_count = player_count,
    player_id = 0
)
    
ai_1: autoPlayer.HexAgent = autoPlayer.RandomHexAgent(
    size = game_size,
    player_count = player_count,
    player_id = 1
)
    

In [ ]:
# Have them play eachother
literalHistory: jable.JyFrame = autoPlayer.runHexathello_withAgents(
    agents = [ai_0, ai_1],
    size = game_size,
    logging_level = 0
)

In [ ]:
# a literal history shows the state at turn from the board's point of view. However, to do machine learning,
#  we need to shift the encoding of state as if it were from player 0's point of view. Otherwise, we would need
#  a different network for each player index; 2, 3 or 6 times as many

povHistory: jable.JyFrame = autoPlayer.povHistory_from_literalHistory(
    literalHistory
)

printing.prettyprint( povHistory )

In [1]:
# Save this to the disk to be used for future learning. We encode the state and play vectors as integers
povHistory_encoded: jable.JyFrame = autoPlayer.history_asInt(
    povHistory
)

# Test we encode and decode appropriately
if True:
    povHistory_decoded: jable.JyFrame = autoPlayer.history_fromInt(
        povHistory_encoded
    )
    _povHistory_i: dict[ str, any ] = {}
    _povHistory_decoded_i: dict[ str, any ] = {}
    
    assert len( povHistory ) == len( povHistory_decoded )

    for i in range( len( povHistory ) ):
        _povHistory_i: dict = povHistory[i]
        _povHistory_decoded_i: dict = povHistory_decoded[i]

        if not len( _povHistory_i["board_state"] ) == len( _povHistory_decoded_i["board_state"] ):
            print("# Incongruity in len 'board_state' at row index={}".format(i))
            print( _povHistory_i )
            print( _povHistory_decoded_i )
            raise Exception("Len Incongruity")
        #

        if not np.all(
            _povHistory_i["board_state"] == _povHistory_decoded_i["board_state"]
        ):
            print("# Incongruity in 'board_state' at row index={}".format(i))
            print( _povHistory_i )
            print( _povHistory_decoded_i )
            raise Exception("Incongruity")
        #

        if not len( _povHistory_i["player_action"] ) == len( _povHistory_decoded_i["player_action"] ):
            print("# Incongruity in len 'player_action' at row index={}".format(i))
            print( _povHistory_i )
            print( _povHistory_decoded_i )
            raise Exception("Len Incongruity")
        #

        if not np.all(
            _povHistory_i["player_action"] == _povHistory_decoded_i["player_action"]
        ):
            print("# Incongruity in 'player_action' at row index={}".format(i))
            print( _povHistory_i )
            print( _povHistory_decoded_i )
            raise Exception("Incongruity")
        #    
    #/for i in range( len( povHistory ) )
    
    del _povHistory_decoded_i
    del _povHistory_i
    del povHistory_decoded
    
    print("# Encode/decode success")
#/if True

# Prepare the save location for future learning
histories_dir: str = path.join(
    'data',
    'histories'
)
assert path.isdir(
    histories_dir
)

test_data_name: str = 'random_size-{}_players-{}'.format(
    game_size, player_count
)

# Make sure it's a new file
while path.isfile(
    path.join(
        histories_dir,
        '{}.json'.format( test_data_name )
    )
):
       test_data_name = '{}_TEMP'.format( test_data_name )
#

test_data_path: str = path.join(
    histories_dir,
    '{}.json'.format( test_data_name )
)
povHistory_encoded.write_file(
    test_data_path
)

# Read the data from disk to learn from
# (We could use povHistory directly in this particular case)
povHistory_fromDisk: jable.JyFrame = jable.read_file(
    test_data_path
)

# Decode the state and play vectors from integers to numpy arrays
povHistory_decoded: jable.JyFrame = autoPlayer.history_fromInt(
    povHistory_fromDisk
)

# We want to train a Keras Neural Network on the data we have.
# The input size is the length of a state vector
# The output size is the length of the play vector
# Take both from the first row
input_size: int = len( povHistory_decoded[0,'board_state'] )
output_size: int = len( povHistory_decoded[0, 'player_action'] )

# The `KerasHexAgent` subclass of `HexAgent` has a `brain` property; this is the neural network
# We could in fact us any objects conforming to the `PredictionModel` protocol, which has methods:
#   - fit()
#   - predict()
#   - call()
#
# We're going to train it on the random choices
import tensorflow as tf
brain_input = tf.keras.layers.Input(
    shape = (input_size,),
    name = 'keras_tensor'
)

# Get creative with architecture
brain_next = tf.keras.layers.Dense(
    input_size**2,
    activation = 'relu'
)( brain_input )

brain_next = tf.keras.layers.Dense(
    input_size**2,
    activation = 'relu'
)

brain_output = tf.keras.layers.Dense(
    output_size,
    activation = 'sigmoid'
)

brain_model: tf.keras.Model = tf.keras.Model(
    brain_input,
    brain_output
)
    
# Choose your learning rate and optimizer. Adam is probably good for the latter.
# You most likely want Binary Cross Entropy
brain_model.compile(
    optimizer = tf.keras.optimizers.Adam(
        learning_rate = 0.005
    ),
    loss = tf.keras.losses.BinaryCrossentropy()
)

# Init the AI Agent
ai_keras: autoPlayer.KerasHexAgent = autoPlayer.KerasHexAgent(
    size = game_size,
    player_count = player_count,
    brain = brain_model,
    player_id = None,
    ai_id = 'Keras_alpha_0'
)


assert path.isfile( test_data_path )

os.remove( test_data_path )
assert not path.isfile( test_data_path )

Early end, no move for any player
RESULT: 21 - 21; Tie among 0, 1
# Game done
{'winner': None, 'turn_index': 36, 'size': 5, 'game_complete': True, 'empty_count': 19, 'player_count': 2, 'current_player': 0, 'scores': [21, 21]}
# Encode/decode success
